## Image classification with deep learning methods.

-- Description --

When you train the network, it is recommended to use the GPU resources of your computer.
This will help you to learn the "know how" of setting up a working Python environment on your computer.
In the case of unavailable Nvidia hardware or problems with your Python environment you can use Google Colab.
Please go to the menu, Runtime - Change runtime type, and select **GPU** as the hardware accelerator.
Although you used your computer successfuly it is highly recommended to give a try to Google Colab environment.


In [15]:
# Import libraries
# These libraries should be sufficient for this Practice.
# However, if any other library is needed, please install it by yourself.

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torch.utils.data as data
import numpy as np
import time
import os
import random
import matplotlib.pyplot as plt
from matplotlib import colors
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import tensorflow as tf

!pip install medmnist
import medmnist
from medmnist import *

In [16]:
# Parameters
NUM_EPOCHS = 20
BATCH_SIZE = 28
lr = 0.0001
DOWNLOAD_OK = True
data_flag = 'bloodmnist'
im_size = 3*28*28
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
num_classes = len(info['label'])
N_IMAGES = 1000
data_labels = info['label']

# Tupla que contiene los valores asociados a los parámetros mostrados anteriormente,
# con el fin de mejorar la organización de dichos parámetros.
parameters = {"num_epochs": NUM_EPOCHS, "batch_size": BATCH_SIZE, "lr": lr, "download_ok": DOWNLOAD_OK,
              "data_flag": data_flag, "im_size": im_size,"info_task": task, "n_channels": n_channels,
              "num_classes": num_classes,"n_images":N_IMAGES, "data_labels": data_labels}

# Preprocesado de datos mediante la definición de la transformación de datos
def preprocessing_data(parameters):
  data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
  ])

  full_train_dataset = BloodMNIST(split="train", transform=data_transform, download=True)
  #full_valid_dataset = BloodMNIST(split="val", transform=data_transform, download=True)
  full_test_dataset = BloodMNIST(split="test", transform=data_transform, download=True)

  idx_train = np.random.choice(len(full_train_dataset),size=800,replace=False)
  train_dataset = [full_train_dataset[i] for i in idx_train]

  #idx_valid = np.random.choice(len(full_valid_dataset),size=100,replace=False)
  #valid_dataset = [full_valid_dataset[i] for i in idx_valid]

  idx_test = np.random.choice(len(full_test_dataset),size=200,replace=False)
  test_dataset = [full_test_dataset[i] for i in idx_test]


  train_loader = data.DataLoader(dataset=train_dataset, batch_size=parameters["batch_size"], shuffle=True)
  #valid_loader = data.DataLoader(dataset=valid_dataset, batch_size=parameters["batch_size"], shuffle=False)
  test_loader = data.DataLoader(dataset=test_dataset, batch_size=parameters["batch_size"], shuffle=True)

  #return train_loader, valid_loader, test_loader
  return train_loader, test_loader

train_loader,test_loader = preprocessing_data(parameters)

Using downloaded and verified file: /root/.medmnist/bloodmnist.npz
Using downloaded and verified file: /root/.medmnist/bloodmnist.npz


In [ ]:
## Your code

# Función que permite visualizar los aspectos fundamentales sobre cada
# dataset que se haya descargado/cargado previamente.
def dataset_visualizer(dataset,length_montage):
  for i in range(0,length_montage*length_montage):
    img = dataset[i][0]
    label = str(dataset[i][1]).replace('[','').replace(']','')
    figure = plt.figure(figsize=(2,2))
    plt.imshow(img.permute(1,2,0))
    plt.title(data_labels[label])
    plt.axis("off")
  plt.show()

# Visualizador de las imágenes a través de un pipeline DataLoader
def dataloader_visualizer(data_loader,num_batches):
  for batch_idx, (features, labels) in enumerate(data_loader):
      if batch_idx >= num_batches:
        break
      for i in range(len(features)):
        img = features[i].squeeze()
        label = str(labels[i]).replace('tensor([','').replace('])','')
        plt.figure(figsize=(2,2))
        plt.title(label)
        plt.imshow(img.permute(1,2,0))
        plt.axis('off')
        plt.show()

def net_model_visualizer(net_model):
  print(net_model)
  print('Total Parameters:',
       sum([torch.numel(p) for p in net_model.parameters()])
  )
  print('Trainable Parameters:',
       sum([torch.numel(p) for p in net_model.parameters() if p.requires_grad])
  )

dataloader_visualizer(train_loader,1)
#dataloader_visualizer(valid_loader,1)
dataloader_visualizer(test_loader,1)

In [17]:
# Define a simple CNN model

class Net(nn.Module):
    def __init__(self, n_channels, num_classes, im_size):
        super(Net, self).__init__()
        #Define the desired deep learning model
        #Your code

        self.conv1 = nn.Conv2d(n_channels,28,kernel_size=(3,3),stride=1,padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(28,28,kernel_size=(3,3),stride=1,padding=1)
        self.relu2 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2,2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*14*14,512)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(512,num_classes)

        #End your code

    def forward(self, x):
        #Your code

        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.pool1(x)
        x = self.flatten(x)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)

        return x

        #End your code

In [18]:
# Train the model
def train_epoch(model, train_loader, optimizer, criterion, task):

    correct = 0
    total_loss = 0
    total_samples = 0

    batch_loss = []

    model.train()

    for inputs,targets in train_loader:
        optimizer.zero_grad()
        with torch.no_grad():
          outputs = model(inputs)

          loss = criterion(outputs,targets.float())

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          batch_loss.append(loss.item())

          # Calcula el número de predicciones correctas
          preds = torch.round(outputs)
          correct += (preds == targets).sum().item()
          total_samples += targets.size(0)

    return {
        "train_acc": correct / total_samples,
        "batch_loss": batch_loss
    }

def validate_epoch(model,test_loader,criterion,task):
    correct = 0
    total_loss = 0
    total_samples = 0

    batch_loss = []

    model.eval()
    for inputs, targets in (train_loader):
        outputs = model(inputs)
        loss = criterion(outputs,targets)

        preds = torch.round(outputs)
        correct += (preds == targets).sum().item()
        total_samples += targets.size(0)

        batch_loss.append(loss.item())

    return {
        "valid_acc": correct / total_samples,
        "batch_loss": batch_loss
    }




def model_training(train_loader,test_loader,parameters):
  model = Net(n_channels=parameters["n_channels"],
              num_classes=parameters["num_classes"],
              im_size=parameters["im_size"])

  optimizer = torch.optim.Adam(model.parameters(),lr=lr)
  criterion = torch.nn.MSELoss()

  for epoch in range(parameters["num_epochs"]):
    train_history = train_epoch(model,train_loader,optimizer,criterion,task)
    valid_history = validate_epoch(model,test_loader,criterion,task)
    print(train_history)
    print(valid_history)



model_training(train_loader,test_loader,parameters)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([28, 1])) that is different to the input size (torch.Size([28, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([16, 1])) that is different to the input size (torch.Size([16, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


{'train_acc': 0.8675, 'batch_loss': [21.98282814025879, 18.476560592651367, 19.723663330078125, 23.173051834106445, 19.274524688720703, 15.26109790802002, 12.398507118225098, 14.08562183380127, 17.316814422607422, 17.939159393310547, 10.49339485168457, 11.604945182800293, 15.886347770690918, 9.70026683807373, 9.933979988098145, 10.1796875, 10.836440086364746, 9.534601211547852, 7.1734514236450195, 8.852312088012695, 6.829930305480957, 8.80164623260498, 6.034141540527344, 4.3272786140441895, 4.728830814361572, 5.536717414855957, 4.6035237312316895, 5.352776527404785, 6.741387367248535]}
{'valid_acc': 0.4025, 'batch_loss': [4.177454948425293, 5.183654308319092, 5.094069004058838, 5.724249839782715, 5.804703235626221, 6.439949989318848, 3.830047130584717, 5.639470100402832, 6.769477367401123, 4.608656406402588, 6.352260589599609, 6.714452266693115, 4.49691104888916, 5.065406799316406, 8.408820152282715, 4.832237243652344, 5.9265618324279785, 7.0522894859313965, 3.6829378604888916, 5.41688

#Evaluation

Finally, implement the evaluation of the object clasification task. You can implement any metric you want, though the most common are accuracy and AUC (one class against all for the multiclass task). You can use torch.no_grad() for speeding up predictions when no gradients are needed.

How do you compare with the MedMNIST benchmarks?

In [ ]:
# Evaluation

# Your code
# Función que permite calcular aquellas métricas que resultan
# más interesantes para la evaluación del modelo de predicción
def model_test_metrics(model,test_loader,criterion,task):

